In [2]:
# Run the paired end
!shi7 -i ../tests/testfq/GA01NEX/ -o ./data/pe --adaptor Nextera

In [71]:
# Run the single end
!shi7 -i ../tests/testfq/GA01NEX/ -o ./data/se --adaptor Nextera --flash False

In [9]:
# Gather a dictionary of all paired ends
def read_fasta(fh):
    """
    :return: tuples of (title, seq)
    """
    title = None
    data = None
    for line in fh:
        if line[0] == ">":
            if title:
                yield (title.strip(), data)
            title = line[1:]
            data = ''
        else:
            data += line.strip()
    if not title:
        yield None
    yield title.strip(), data

In [48]:
output_file = "data/combined_seqs.all.fna"

In [52]:
paired_success = set()

with open("./data/pe/combined_seqs.fna", "r") as inf:
    with open(output_file, "w") as outf_fasta:
        gen_fasta = read_fasta(inf)
        for title, seq in gen_fasta:
                # Grab the name of the sequence
                paired_success.add(title.split()[0])
                outf_fasta.write('>%s\n%s\n' % (title, seq))

In [69]:
num_not_stitched = 0
with open("./data/se/combined_seqs.fna", "r") as inf:
    with open(output_file, "a") as outf_fasta:
        gen_fasta = read_fasta(inf)
        for title, seq in gen_fasta:
            title_check = title.split()[0].replace(".R1", "").replace(".R2", "")
            if not title_check in paired_success:
                num_not_stitched += 1
                print(title)
                outf_fasta.write('>%s\n%s\n' % (title, seq))

In [73]:
def combine_stitched_se(output_file, pe_inf, se_inf):
    paired_success = set()

    with open(pe_inf, "r") as inf:
        with open(output_file, "w") as outf_fasta:
            gen_fasta = read_fasta(inf)
            for title, seq in gen_fasta:
                    # Grab the name of the sequence
                    paired_success.add(title.split()[0])
                    outf_fasta.write('>%s\n%s\n' % (title, seq))
    
    num_not_stitched = 0
    with open(se_inf, "r") as inf:
        with open(output_file, "a") as outf_fasta:
            gen_fasta = read_fasta(inf)
            for title, seq in gen_fasta:
                title_check = title.split()[0].replace(".R1", "").replace(".R2", "")
                if not title_check in paired_success:
                    num_not_stitched += 1
                    print(title)
                    outf_fasta.write('>%s\n%s\n' % (title, seq))
    print("Number not stitched %d" % num_not_stitched)

In [76]:
se_inf = "./data/se/combined_seqs.fna"
pe_inf = "./data/pe/combined_seqs.fna"

In [77]:
combine_stitched_se(output_file, pe_inf, se_inf)

Number not stitched 0
